<a href="https://colab.research.google.com/github/chetools/CHE4071_Fall2024/blob/main/TanksInSeriesLT_vs_NonLinearModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

In [74]:
from importnb import Notebook
with Notebook():
    from che5 import sim, pid, TF1, TF2, shift

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

from plotly.subplots import make_subplots

import sympy
from sympy.abc import s
from sympy import exp, Symbol, simplify
import scipy as sp
import scipy.signal as sig
from scipy.integrate import solve_ivp
from scipy.optimize import root, minimize

In [84]:
Ac = 0.9
Ah = 0.2
g = 9.8

def rhs(t, v, qin):
    qin1, qin2 = qin
    h1, h2, h3 = v
    q12 = Ah*jnp.sqrt(2*g*h1)
    q23 = Ah*jnp.sqrt(2*g*h2)
    q3 = Ah*jnp.sqrt(2*g*h3)

    dh1  = (qin1 - q12)/Ac
    dh2 = (q12 - q23 + qin2)/Ac
    dh3 = (q23 - q3 )/Ac
    return jnp.array([dh1, dh2, dh3])

In [85]:
qin_initial_ss = np.array([0.5, 0.1])
h_initial_ss= minimize(lambda x: 0., np.array([10., 10., 10.]), constraints=dict(type='eq', fun = lambda h: rhs(0.,h, qin_initial_ss)),
         bounds = [[0,np.inf]]*3).x

In [86]:
A=jax.jacobian(rhs, 1)(0., h_initial_ss, qin_initial_ss)
B=jax.jacobian(rhs, 2)(0., h_initial_ss, qin_initial_ss)

In [88]:
TF=((s*sympy.eye(3)-sympy.Matrix(A))**-1) @ sympy.Matrix(B)
TF

Matrix([
[1.11111111111111*(1.0*s**2 + 1.45185185187319*s + 0.5269684499469)/(1.0*s**3 + 2.32296296299412*s**2 + 1.79169272981514*s + 0.459048071958921),                                                                                                                                                0],
[        1.11111111111111*(0.871111111120935*s + 0.632362139934119)/(1.0*s**3 + 2.32296296299412*s**2 + 1.79169272981514*s + 0.459048071958921), 1.11111111111111*(1.0*s**2 + 1.59703703705753*s + 0.632362139934119)/(1.0*s**3 + 2.32296296299412*s**2 + 1.79169272981514*s + 0.459048071958921)],
[                                                 0.702624599926799/(1.0*s**3 + 2.32296296299412*s**2 + 1.79169272981514*s + 0.459048071958921),           1.11111111111111*(0.725925925936593*s + 0.632362139934119)/(1.0*s**3 + 2.32296296299412*s**2 + 1.79169272981514*s + 0.459048071958921)]])

In [96]:
t, h3_dev = sim(TF[2,0], lambda t: 1.)
res = solve_ivp(lambda t, v: rhs(t, v, qin_initial_ss + np.array([1.,0.])), (0, t[-1]), h_initial_ss, method='Radau', dense_output=True)

In [101]:
fig=make_subplots()
fig.add_scatter(x=t,y=h3_dev+h_initial_ss[2])
fig.add_scatter(x=t, y= res.sol(t)[2])

In [ ]:
H3 = TF[2,0] U1 + TF[2,1] U2